### AI 면접 모의고사 및 피드백


#### ※ 애플리케이션 프로세스

1.본인 직무 선택 후, 면접 질문 생성 요청

2.OpenAI (Chat)이 면접 질문 생성

3.TTS로 질문 음성 재생

4.사용자가 마이크를 통해 답변을 녹음

5.STT로 답변 음성을 텍스트로 변환

6.AI Language로 답변의 긍정/부정 감정, 주요 키워드를 분석

7.OpenAI (Chat)모델이 AI Language의 결과를 바탕으로, 답변 텍스트 내용의 논리성, 분석된 감정 상태, 사용된 키워드를 종합하여 피드백 텍스트를 생성 및 출력


In [ ]:
import gradio as gr
import os
import time
import tempfile
from openai import AzureOpenAI
import requests
from dotenv import load_dotenv
# .env 환경변수 로드
load_dotenv()

# ==============================================================================
# 1. API 설정 및 클라이언트 초기화 (사용자 변경 필요 부분)
# ==============================================================================

# 1-1. OpenAI 설정
openai_client = AzureOpenAI(
    api_key=os.getenv("AZURE_OAI_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT")
)

# 1-2. Azure AI Speech 설정 (TTS, STT)
SPEECH_KEY = os.getenv("STT_OCP_APIM_SUB_KEY")
SPEECH_REGION = "eastus"  # 예: eastus, korea

# 1-3. Azure AI Language 설정 (감정/키워드 분석)
LANGUAGE_ENDPOINT = "https://8ai022-language.cognitiveservices.azure.com/language/:analyze-text?api-version=2023-04-01"
LANGUAGE_KEY = os.getenv("AZURE_LANGUAGE_KEY")


# 면접 질문 생성
def generate_questions_logic(career_type, years, job_major, job_middle, job_minor, salary, kind):
    """
    [Step 1] OpenAI를 사용하여 면접 질문 생성
    """
    
    # 경력 정보 텍스트 정리
    career_info = "신입"
    if career_type == "경력":
        career_info = f"경력 ({years}년차)"

    try:
        endpoint = "https://8ai022-openai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"

        headers = {
            'api-key' : os.getenv("AZURE_OAI_KEY"),
            'Content-Type' : 'application/json'
        }

        payload = {
            "messages": [
                {"role": "system", "content": f"당신은 지원자가 지원한 직무의 전문 채용 면접관입니다. 지원자의 직무를 바탕으로 경력 연차에 따른 심도 있는 {kind} 면접 질문 1개를 생성해주세요."}, 
                {"role": "user", "content": f"""[지원자 정보]
                                                - 경력 구분: {career_info}
                                                - 직무: {job_major} > {job_middle} > {job_minor}
                                                - 희망 연봉 범위: {salary}
                                                
                                                [주의 사항]
                                                - 반드시 질문 내용만 줄바꿈으로 구분하여 출력해주세요. 
                                                - 번호(1., 2.)는 붙이지 마세요. 
                                                - 중복되는 질문이 발생하지 않도록 다양한 질문을 생각하고 랜덤으로 1개를 생성하세요.
                                                - 질문에 직무에 대한 텍스트는 제외하세요.
                                                - 애매모호하거나 포괄적인 질문을 하지 말고, 아래 출력 형식 예시와 같이 직무에 대한 직관적인 질문만을 생성하세요.
                                                - 연차에 따라 질문의 난이도를 다르게 설정하세요.

                                                [기술 면접 출력 형식 예시 - JAVA 개발자]
                                                로그인 회원 정보를 세션에 저장하는 방식과, Token을 발급하는 방식에서 보안적 관점의 차이점을 설명해주세요.

                                                [기술 면접 출력 형식 예시 - 인사담당]
                                                회사 비용 절감에 따른, 구조조정이 불가피한 상황입니다. 모든 직원의 역량이 우수한 상황인데, 어떠한 직원을 어떠한 기준으로 차출하여 희망퇴직을 시킬것인가요?
                                                
                                                [인성 면접 출력 형식 예시]
                                                현재 지원자의 경력은 {career_info}인데, 그에 맞는 역략을 보유하고 있다고 생각하시나요? 그렇다라고 한다면 어떠한 역량을 가지고 있나요?

                                                """}

                                                
            ],
            "max_tokens": 1000,
            "model": "gpt-4o-mini"
        }

        response = requests.post(endpoint, headers=headers, json=payload)

        result_text = ''

        if response.status_code == 200 :
            response_json = response.json()
            result_text = response_json['choices'][0]['message']['content']
            print(f"response_json : {response_json}")
        else : 
            print(f'OpenAI Http Request Error : {response.status_code}')


        # 텍스트를 줄바꿈 기준으로 분리하여 리스트로 반환
        questions = [q.strip() for q in result_text.split('\n') if q.strip()]
        
        return questions[:int(3)] # 혹시 더 많이 생성되었을 경우를 대비해 자름
    
    except Exception as e:
        print(f"질문 생성 중 오류: {e}")
        return ["오류가 발생하여 기본 질문을 제공합니다."] * int(3)


# 면접 질문 텍스트 => 오디오로 변환
def text_to_speech_file(text):
    """
    [Step 2] Azure TTS: 텍스트를 오디오 파일로 변환
    """
    endpoint = "https://eastus.tts.speech.microsoft.com/cognitiveservices/v1"

    headers = {
        "Content-Type" : "application/ssml+xml",
        "Ocp-Apim-Subscription-Key" : os.getenv("STT_OCP_APIM_SUB_KEY"),
        "X-Microsoft-OutputFormat" : "audio-16khz-128kbitrate-mono-mp3"
    }

    # 3. Body : SSML
    body = f"""
    <speak version='1.0' xml:lang='ko-KR'>
        <voice name="ko-KR-SoonBokNeural" style="unfriendly">
            {text}
        </voice>
    </speak>
    """

    # request Audio => Text
    response = requests.post(endpoint, headers=headers, data=body)

    if response.status_code != 200 :
        print(f"Http Request Error {response.status_code}")
        return None
    
    
    # Hex => mp3 파일로 변환
    import datetime
    now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file_path = f"files/tts_{now}output.mp3"
    with open(output_file_path, "wb") as output_audio :
        output_audio.write(response.content)
    
    return output_file_path

# 답변 음성 => 텍스트 변환
def speech_to_text_file(audio_path):
    """
    [Step 3] Azure STT: 사용자의 답변 오디오 파일을 텍스트로 변환
    """
    # 1. Endpoint
    endpoint = "https://eastus.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1?language=ko-KR&format=detailed"

    # 2. Header
    headers = {
        "Content-Type" : "audio/wav",
        "Ocp-Apim-Subscription-Key" : SPEECH_KEY
    }

    # 3. Body : Audio file => binary
    with open(audio_path, "rb") as audio_file : 
        audio_data = audio_file.read()

    body = audio_data

    # request Audio => Text
    response = requests.post(endpoint, headers=headers, data=body)

    if response.status_code != 200 :
        print(f"Http Error {response.status_code} : {response.reason}")
        return None
    
    import json

    # String => Dictionary
    result_json = json.loads(response.text)

    if result_json.get("RecognitionStatus") != "Success" : 
        print("STT Response Error ", result_json.get("RecognitionStatus"))
        return None
    
    # return Audio => Text
    return result_json.get("DisplayText")

# STT 답변 음성 텍스트 변환
# AI Language (감정, 키워드 분석)
# OpenAI (최종 피드백 생성)
def analyze_answer_logic(audio_file, question_text):
    
    if audio_file is None:
        return "⚠️ 답변이 녹음되지 않았습니다. 녹음 후 다시 시도해주세요."

    # 1. 답변 음성 => Text 변환
    user_answer_text = speech_to_text_file(audio_file)
    if not user_answer_text or user_answer_text == "(음성 인식 실패 또는 내용 없음)":
        return "⚠️ 음성을 인식할 수 없습니다. 다시 명확하게 말씀해주세요."

    # 2. Azure AI Language 분석

    headers = {
        "Content-Type" : "application/json",
        "Ocp-Apim-Subscription-Key" : LANGUAGE_KEY
    }

    body = {
        "kind": "SentimentAnalysis",
        "analysisInput": {
            "documents": [
                {
                    "id": "id__9000",
                    "text": user_answer_text,
                    "language": "ko"
                }
            ]
        },
        "parameters": {
            "opinionMining": False
        }
    }

    response = requests.post(LANGUAGE_ENDPOINT, headers=headers, json=body)

    if response.status_code != 200 :
        print(f"Http Request Error {response.status_code}")

    import json
    result_json = json.loads(response.text)

    # 문서 (Document) 데이터 추출
    document = result_json['results']['documents'][0]

    # 1. 전체 답변 텍스트 재구성
    full_text_list = [s['text'] for s in document['sentences']]
    full_answer_text = "".join(full_text_list).strip()

    # 2. 전체 감정 추출
    overall_sentiment = document['sentiment']

    # 3. 문장별 상세 분석 내용 구축
    sentence_details = []

    for i, sentence in enumerate(document['sentences']):
        # 감정 값과 신뢰도(Confidence) 추출 및 퍼센트 포맷
        sentiment_type = sentence['sentiment']
        confidence = sentence['confidenceScores'][sentiment_type] * 100
        
        # 상세 정보를 문자열로 포맷
        detail_line = (
            f"- **문장 {i+1}:** \"{sentence['text'].strip()}\"\n"
            f"  - **감정:** {sentiment_type.upper()} ({confidence:.1f}%)"
        )
        sentence_details.append(detail_line)

    # 4. LLM에 전달할 최종 프롬프트 문자열 구성 (Markdown 형식)

    # 데이터 구조화
    data_section = f"""
    ---
    ## 💡 AI 분석 기반 피드백 요청 데이터
    ** [0. 면접 질문] **
    {question_text}

    **[1. 지원자 답변 원본 텍스트]**
    {full_answer_text}


    **[2. 전체 감정 분석 결과]**
    - **전반적 감정:** {overall_sentiment.upper()}

    **[3. 문장별 상세 감정 변화]**
    {sentence_details}
    ---
    ## 📝 피드백 작성 가이드라인

    1. **내용 평가:** 답변의 논리적 흐름과 직무 연관성 키워드 사용의 적절성을 평가해 주세요.
    2. **감정/태도 평가:** 감정 상태와 키워드를 참고하여 태도나 어휘 선택에 대한 조언을 해주세요. 단, 답변에 감정에 대한 텍스트는 제외하세요.
    3. **최종 요약:** 답변에서 좋았던 점과 개선해야 할 점을 요약해 주세요. 없다면 생성하지 않아도 됩니다.
    
    ## 주의사항
    1. 공정성, 객관성 강화 : 주관적인 판단은 줄이고, 동일한 기준으로 일관된 평가를 제공하세요.
    2. 답변의 논리성, 문제 해결력, 의사소통 능력, 자신감, 정서적 안정성도 평가에 참고하세요.
    """

    # 최종 프롬프트 결합
    feedback_prompt = f"{data_section}\n\n**[피드백 시작]**\n"
        
    endpoint = "https://8ai022-openai.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview"

    headers = {
        'api-key' : os.getenv("AZURE_OAI_KEY"),
        'Content-Type' : 'application/json'
    }

    payload = {
        "messages": [
            {"role": "system", "content": """당신은 지원자의 답변을 전문적으로 평가하는 면접 코치입니다. 
                                             아래 제공된 답변 텍스트와 감정 분석 결과를 바탕으로, 지원자에게 내용, 전달력, 자신감 측면에서 개선점을 포함한 상세한 피드백을 한국어로 작성해 주세요.

                                          """}, 
            {"role": "user", "content": feedback_prompt}
        ],
        "max_tokens": 1000,
        "model": "gpt-4o-mini"
    }

    response = requests.post(endpoint, headers=headers, json=payload)

    result_text = ''

    if response.status_code == 200 :
        response_json = response.json()
        result_text = response_json['choices'][0]['message']['content']
    else : 
        print(f"Open AI Http Request Error {response.status_code}")

    return result_text

# ==============================================================================
# 3. Gradio UI 구성
# ==============================================================================

# CSS 정의: 브라우저 너비의 60%로 메인 컨테이너 너비 제한
custom_css = """
/* 전체 Gradio 컨테이너의 최대 너비를 60%로 설정 */
.gradio-container {
    width: 70% !important; 
    margin: 0 auto; /* 가운데 정렬 */
}
/* 모바일 화면에서는 너비를 100%로 유지 */
@media (max-width: 768px) {
    .gradio-container {
        max-width: 100% !important; 
    }
}
"""
JOB_CATEGORIES = {
    # ====================================================================
    # 1. IT/개발 (정보통신 및 소프트웨어 개발)
    # ====================================================================
    "IT/개발": {
        "백엔드 개발": [
            "Java 개발자", "Python 서버 개발자", "Node.js 개발자", "Go/Serverless 개발", "PHP/Laravel 개발",
            "C#/ASP.NET 개발", "RDBA (관계형DB 관리)", "DevOps 엔지니어", "MSA/Cloud 아키텍트", "WAS/Web 서버 엔지니어"
        ],
        "프론트엔드 개발": [
            "ReactJS 개발자", "Vue.js 개발자", "AngularJS 개발자", "웹 퍼블리셔", "UI/UX 개발자",
            "Web/3D 개발", "JavaScript 전문가", "HTML/CSS 전문가", "TypeScript 개발", "크로스 브라우징 전문가"
        ],
        "AI/데이터": [
            "데이터 사이언티스트", "머신러닝 엔지니어(ML)", "딥러닝 엔지니어", "빅데이터 엔지니어", "데이터 분석가",
            "데이터 아키텍트", "자연어 처리(NLP)", "컴퓨터 비전(CV)", "데이터베이스 관리(DBA)", "Business Intelligence (BI)"
        ],
        "모바일 개발": [
            "iOS Swift/Objective-C", "Android Kotlin/Java", "Flutter 개발", "React Native 개발", "Unity/모바일 게임 개발",
            "모바일 UI/UX", "모바일 테스트 엔지니어", "모바일 백엔드 연동", "웨어러블 앱 개발", "크로스 플랫폼 개발"
        ],
        "웹/앱 기획(PM/PO)": [
            "서비스 기획자", "프로덕트 매니저(PM)", "프로덕트 오너(PO)", "UI/UX 기획", "백오피스/관리시스템 기획",
            "B2B 서비스 기획", "플랫폼 기획", "시스템 분석가(SA)", "비즈니스 분석가(BA)", "기술 전략 기획"
        ],
        "보안/QA": [
            "정보보호 전문가", "침해사고 분석(CERT)", "보안 솔루션 개발", "모의해킹/취약점 분석", "QA 엔지니어",
            "소프트웨어 테스트", "성능 테스트(APM)", "보안관제", "IT 감사", "시스템 보안 엔지니어"
        ],
        "ERP/시스템 운영": [
            "SAP 컨설턴트", "ERP 구축/운영", "SCM/MES 시스템 관리", "PLM/PDM 시스템 관리", "DW/BI 시스템",
            "IT 인프라 구축", "시스템 유지보수", "시스템 엔지니어(SE)", "클라우드 엔지니어(AWS/Azure)", "네트워크 엔지니어"
        ],
        "하드웨어/임베디드": [
            "임베디드 소프트웨어 개발", "펌웨어 개발", "PCB 설계", "회로 설계", "H/W 엔지니어",
            "센서 개발", "IoT 개발", "통신/네트워크 H/W", "SoC 설계", "ASIC/FPGA 설계"
        ],
        "게임 개발": [
            "클라이언트 프로그래머", "서버 프로그래머", "게임 기획자", "게임 QA", "게임 디자이너",
            "엔진 프로그래머", "모바일 게임 개발", "VR/AR 게임 개발", "게임 아티스트", "이펙트 디자이너"
        ],
        "통신/네트워크": [
            "네트워크 엔지니어", "통신망 구축", "데이터 센터(IDC) 운영", "VoIP/IPTV 기술", "5G/6G 기술",
            "네트워크 보안", "L2/L3 스위치 관리", "라우터 관리", "서버 관리자", "망 관리/운용"
        ]
    },
    # ====================================================================
    # 2. 경영/기획 (전략, 인사, 재무 등)
    # ====================================================================
    "경영/기획": {
        "전략/사업 기획": [
            "경영 전략 수립", "신사업 기획/검토", "사업 개발(BD)", "M&A/투자 기획", "해외 사업 개발",
            "경영 분석", "전사 기획", "사업 타당성 분석", "PMO(프로젝트 관리)", "비전 및 미션 수립"
        ],
        "인사/HR": [
            "인사 기획", "채용(Recruiter)", "교육/개발(TD)", "보상/승진(C&B)", "노무 관리",
            "HRBP(파트너)", "인사 시스템(HRIS) 운영", "조직 문화 관리", "평가/성과 관리", "인력 운영"
        ],
        "재무/회계": [
            "재무 회계", "관리 회계", "세무", "자금/Cash Flow 관리", "IR(투자자 관계)",
            "내부 감사", "원가 회계", "예산 관리/편성", "결산 업무", "회계 감사 대응"
        ],
        "총무/법무": [
            "총무/자산 관리", "시설 관리", "문서/기록 관리", "법무(변호사/계약 검토)", "컴플라이언스",
            "지적 재산권(IP) 관리", "소송 대응", "대관 업무", "보안/의전", "업무 프로세스 개선(BPR)"
        ],
        "금융/자산 운용": [
            "투자 심사", "리스크 관리(RM)", "자산 운용(AM)", "부동산 금융", "기업 금융(IB)",
            "펀드 매니저", "금융 상품 기획", "금융 리서치", "프라이빗 뱅킹(PB)", "보험 상품 설계"
        ],
        "경영 지원": [
            "비서/임원 수행", "사무 보조", "일반 행정", "데이터 관리/집계", "계약 관리",
            "예산 집행 관리", "문서 작성/편집", "회의 운영/지원", "출장/비용 정산", "사무 자동화(RPA)"
        ],
        "해외/수출입 사무": [
            "해외 영업 지원", "수출입 관리", "무역 사무", "통관/관세", "바이어 서류 관리",
            "해외 마케팅 지원", "현지 법인 지원", "국제 계약 관리", "무역 금융 업무", "물류/배송 지원"
        ],
        "비서/의전": [
            "전문 비서", "임원 비서", "수행 기사", "의전 담당", "통역 비서",
            "회의 준비/보조", "일정 관리", "해외 출장 지원", "접대/응대", "내부 커뮤니케이션"
        ],
        "문서/정보 관리": [
            "지식 관리(KM)", "정보 분석", "기록물 관리", "도서관 사서", "데이터 거버넌스",
            "문서 표준화", "아카이빙", "정보 검색 시스템 구축", "출판/편집 관리", "DB 구축/관리"
        ],
        "공공/대외 협력": [
            "연구 기획", "교육 기획/평가", "홍보 기획 지원", "사회 공헌(CSR)", "대외 협력/제휴",
            "주주 관리", "공시 업무", "내부 통제", "윤리 경영", "정부/기관 사업 관리"
        ]
    },
    # ====================================================================
    # 3. 디자인 (시각, 제품, UX/UI 등)
    # ====================================================================
    "디자인": {
        "UX/UI 디자인": [
            "UX 디자이너", "UI 디자이너", "GUI 디자이너", "인터랙션 디자이너", "사용자 경험 리서처",
            "정보 설계(IA)", "프로토타이핑", "디자인 시스템 관리", "모바일 UI 디자인", "웹 접근성 디자인"
        ],
        "웹/모바일 디자인": [
            "웹 디자이너", "모바일 앱 디자인", "반응형 웹 디자인", "랜딩 페이지 디자인", "웹 퍼블리싱 디자인",
            "이커머스 웹 디자인", "플랫폼 UI/UX", "HTML/CSS 코딩", "웹 에이전시 디자인", "포털 디자인"
        ],
        "그래픽/시각 디자인": [
            "브랜드 디자이너(BX)", "아이덴티티 디자인(CI/BI)", "편집 디자인", "인포그래픽", "캐릭터 디자인",
            "일러스트레이터", "타이포그래피", "광고/포스터 디자인", "출판 디자인", "패키지 디자인"
        ],
        "3D/영상/모션": [
            "3D 모델러", "애니메이터", "모션 그래픽 디자이너", "영상 편집", "VFX 전문가",
            "렌더링/시뮬레이션", "VR/AR 콘텐츠 디자인", "게임 배경/캐릭터 모델링", "이펙트 디자이너", "영상 기획/연출"
        ],
        "제품/산업 디자인": [
            "제품 디자이너", "산업 디자이너", "가구/조명 디자인", "금형 설계 디자인", "CMF 디자인",
            "운송 디자인(모빌리티)", "생활용품 디자인", "디자인 엔지니어링", "디자인 전략 수립", "시제품 제작/검토"
        ],
        "공간/환경 디자인": [
            "건축/인테리어 디자이너", "환경 디자인", "조경 디자인", "무대/전시 디자인", "VMD(비주얼 머천다이징)",
            "공간 설계", "전시 기획", "모델하우스 디자인", "시설 디자인", "리모델링 디자인"
        ],
        "패션/텍스타일": [
            "패션 디자이너", "텍스타일 디자이너", "의류/잡화 디자인", "패턴사", "스타일리스트",
            "코디네이터", "트렌드 분석", "패션 MD 지원", "주얼리/액세서리 디자인", "섬유 디자인"
        ],
        "광고/홍보 디자인": [
            "광고 디자이너", "인하우스 디자이너", "브로슈어/카탈로그 제작", "홍보물 제작", "이벤트 디자인",
            "기업 홍보 영상 디자인", "마케팅 콘텐츠 디자인", "소셜 미디어(SNS) 디자인", "ATL/BTL 디자인", "배너/팝업 디자인"
        ],
        "디자인 경영": [
            "디자인 전략 수립", "디자인 리서치", "디자인 컨설팅", "디자인 프로젝트 관리(PM)", "디자인 품질 관리",
            "디자인 표준화", "디자인 교육", "디자인 시스템 구축", "디자인 자산 관리", "디자인 트렌드 분석"
        ],
        "캐릭터/이모티콘": [
            "캐릭터 디자이너", "이모티콘 작가", "웹툰/만화", "굿즈 디자이너", "애니메이션 제작",
            "라이선싱 관리", "IP 개발", "스토리보드", "게임 원화", "컨셉 아티스트"
        ]
    },
    # ====================================================================
    # 4. 마케팅/홍보 (브랜드, 디지털, 광고 등)
    # ====================================================================
    "마케팅/홍보": {
        "디지털 마케팅": [
            "퍼포먼스 마케터", "검색 광고(SEO/SEM)", "데이터 분석 기반 마케팅", "CRM 마케팅", "앱 마케팅(ASO)",
            "웹로그 분석", "마케팅 자동화(MA)", "DMP/CDP 운영", "그로스 해커", "리타겟팅 광고"
        ],
        "콘텐츠/바이럴": [
            "콘텐츠 마케터", "블로그/카페 운영", "유튜브 기획/운영", "소셜 미디어(SNS) 관리", "인플루언서 마케팅",
            "바이럴 마케팅", "브랜드 저널리스트", "라이브 커머스 기획", "숏폼 콘텐츠 제작", "UGC 활용"
        ],
        "브랜드 마케팅": [
            "브랜드 매니저(BM)", "브랜딩 전략", "캠페인 기획", "IMC(통합 마케팅)", "ATL/BTL 마케팅",
            "브랜드 경험(BX)", "브랜드 리서치", "브랜드 가치 관리", "브랜드 스토리텔링", "경쟁사 분석"
        ],
        "홍보/PR/대외": [
            "언론 홍보", "미디어 관리(PPL)", "위기 관리(리스크 PR)", "IR/주주 관계 홍보", "대외 협력",
            "정부/지자체 홍보", "기업 이미지 관리(CI)", "사회 공헌(CSR) 홍보", "사내 커뮤니케이션", "광고 집행 관리"
        ],
        "광고/카피라이터": [
            "카피라이터", "광고 기획(AE)", "광고 매체 기획(Media Planning)", "크리에이티브 디렉터(CD)", "디지털 광고 제작",
            "옥외 광고", "TV/라디오 광고", "스크립터", "광고 효과 분석", "프로모션 기획"
        ],
        "영업 마케팅": [
            "세일즈 마케팅", "영업 지원", "프로모션 기획", "판매 촉진(SP)", "리테일 마케팅",
            "채널 마케팅", "B2B 마케팅", "고객 여정(Customer Journey) 설계", "마케팅 세일즈 연동", "판매 전략 수립"
        ],
        "리서치/분석": [
            "마케팅 리서처", "시장 조사", "소비자 행동 분석", "경쟁사 분석", "설문 조사 기획",
            "트렌드 분석", "데이터 마이닝", "A/B 테스트", "고객 세분화(Segmentation)", "산업 분석"
        ],
        "이벤트/프로모션": [
            "이벤트 기획자", "전시/컨벤션", "프로모션 운영", "부스 디자인/설치", "공연 기획",
            "스폰서십 유치", "체험 마케팅", "팝업 스토어 기획", "연예인/셀럽 섭외", "경품/사은품 관리"
        ],
        "해외 마케팅": [
            "글로벌 마케팅 전략", "해외 광고 집행", "현지 SNS 운영", "다국어 콘텐츠 제작", "해외 시장 리서치",
            "해외 파트너십", "수출 마케팅", "해외 채널 관리", "무역 박람회 참가", "현지 미디어 PR"
        ],
        "CRM/멤버십": [
            "CRM 전문가", "고객 데이터 분석", "멤버십 관리", "고객 로열티 프로그램", "VOC/CS 데이터 분석",
            "이메일 마케팅", "캠페인 성과 측정", "고객 세그먼트 운영", "이탈 방지 전략", "고객 생애 가치(LTV) 분석"
        ]
    },
    # ====================================================================
    # 5. 영업/제휴 (국내, 해외, 기술 영업 등)
    # ====================================================================
    "영업/제휴": {
        "국내 영업": [
            "법인 영업", "일반 영업", "B2B 영업", "B2C 영업", "대리점/채널 관리",
            "신규 거래처 발굴", "프로젝트 영업", "영업 기획/관리", "영업 지원", "판매 대리인 관리"
        ],
        "해외 영업": [
            "수출 영업", "해외 바이어 관리", "무역 영업", "현지 시장 개척", "국제 입찰",
            "OEM/ODM 영업", "해외 지사 관리", "통번역 업무 지원", "해외 파트너십", "글로벌 계약 관리"
        ],
        "기술 영업(SE)": [
            "IT 솔루션 영업", "SW 판매", "HW/장비 영업", "시스템 컨설팅 영업", "산업별 전문 영업",
            "기술 프리젠테이션", "고객 요구사항 분석", "기술 검토", "PoC(개념 증명) 수행", "기술 컨설팅"
        ],
        "제휴/파트너십": [
            "전략적 제휴", "채널 파트너십", "B2B 협력", "콘텐츠 제휴", "MCN/인플루언서 제휴",
            "금융 제휴", "공동 마케팅 제휴", "OEM/ODM 계약 관리", "계약 이행 관리", "신규 제휴처 발굴"
        ],
        "매장 관리/판매": [
            "매장 매니저", "판매 사원", "서비스/세일즈", "바이어/MD 지원", "VMD",
            "재고 관리", "고객 응대", "백화점/마트 관리", "로드샵 운영", "캐셔/POS 관리"
        ],
        "고객 관리(CS)": [
            "고객 상담(인바운드)", "고객 유지(아웃바운드)", "VOC 분석", "CS 매니저", "서비스 교육",
            "콜센터 운영", "채널톡/챗봇 상담", "불만 처리/민원", "고객 만족도 조사(CSAT)", "품질 관리(QAA)"
        ],
        "텔레마케팅": [
            "인바운드 TM", "아웃바운드 TM", "DB 영업", "고객 DB 관리", "상품 설명/안내",
            "해피콜", "재계약 유치", "콜센터 상담", "전화 영업 교육", "스크립트 개발"
        ],
        "영업 관리": [
            "영업 목표 수립", "실적 분석", "판매 예측", "영업 시스템(CRM) 관리", "영업 교육",
            "성과 평가", "인센티브 관리", "영업 채널 전략", "리스크 관리", "영업 보고서 작성"
        ],
        "유통/가맹 관리": [
            "가맹점 관리", "프랜차이즈 운영", "점포 개발", "상권 분석", "슈퍼바이저(SV)",
            "매뉴얼 제작/교육", "물류/재고 관리", "가맹 사업 기획", "매출 분석", "운영 컨설팅"
        ],
        "기타 영업": [
            "금융 상품 판매", "보험 설계사", "제약/의료 기기 영업", "자동차 딜러", "부동산 중개",
            "교육/강의 영업", "여행 상품 판매", "출판물 영업", "회원권 판매", "에너지/화학 영업"
        ]
    },
    # ====================================================================
    # 6. 전문직/특수 (법무, 회계, 연구 등)
    # ====================================================================
    "전문직/특수": {
        "법무/변호사": [
            "변호사", "사내 변호사", "송무", "계약서 검토", "컴플라이언스(준법)",
            "법률 자문", "지적 재산권(IP)", "국제법", "M&A 법률", "금융 법규"
        ],
        "회계/세무사": [
            "공인회계사(CPA)", "세무사", "회계 감사", "법인세", "소득세",
            "재무 컨설팅", "IFRS/K-GAAP", "내부 회계 관리", "세무 조사 대응", "상속/증여세"
        ],
        "금융 전문가": [
            "금융 투자 분석사(CFA)", "자산 관리사(FP)", "투자 심사역", "리스크 관리(RM)", "펀드 매니저",
            "증권 브로커", "금융 상품 기획", "금융 리서치", "채권/외환 딜러", "금융 공학자"
        ],
        "의료/약사": [
            "의사(전문의)", "간호사", "약사", "한의사", "수의사",
            "임상병리사", "방사선사", "물리치료사", "치위생사", "의료 행정"
        ],
        "연구/개발(R&D)": [
            "기초 과학 연구원", "응용 기술 연구원", "신소재 개발", "생명 공학(BT)", "화학 연구",
            "제약 개발", "특허 분석", "국책 과제 수행", "연구 기획", "기술 이전"
        ],
        "통역/번역": [
            "전문 통역사", "번역가", "국제 회의 통역", "기술 문서 번역", "영상 번역",
            "외국어 감수", "현지화(Localization)", "비즈니스 통역", "통역 비서", "언어 교육"
        ],
        "컨설턴트": [
            "경영 전략 컨설턴트", "IT 컨설턴트", "인사 컨설턴트", "재무 컨설턴트", "마케팅 컨설턴트",
            "생산성 컨설턴트", "환경 컨설턴트", "의료 컨설턴트", "비즈니스 프로세스 컨설턴트", "PMO 전문가"
        ],
        "교육/교사": [
            "초/중/고 교사", "대학교수", "입시 컨설턴트", "학원 강사", "기업 교육 강사",
            "이러닝 콘텐츠 개발", "교육 프로그램 기획", "교육 행정", "특수 교육 교사", "평생 교육사"
        ],
        "언론/출판": [
            "기자(취재/편집)", "PD(프로듀서)", "아나운서/MC", "작가(소설/방송)", "에디터",
            "출판 기획", "교정/교열", "편집 디자이너", "도서 영업", "미디어 콘텐츠 제작"
        ],
        "기술/특허 전문가": [
            "변리사", "기술 평가", "특허 출원/등록", "지식 재산권 관리", "기술 라이선싱",
            "기술 마케팅", "기술 문서 작성", "발명 심사", "특허 정보 분석", "기술 이전 전문가"
        ]
    },
    # ====================================================================
    # 7. 생산/품질 (제조, 공정, R&D 등)
    # ====================================================================
    "생산/품질": {
        "생산 관리(PMC)": [
            "생산 계획", "공정 관리", "생산 일정 관리", "생산 효율 분석", "생산성 향상(CAPA)",
            "생산 데이터 분석", "설비 가동률 관리", "자재 수급 계획", "생산 매뉴얼 작성", "안전 보건 관리 지원"
        ],
        "품질 관리(QC/QA)": [
            "품질 기획", "품질 검사(QC)", "품질 보증(QA)", "통계적 공정 관리(SPC)", "품질 시스템 구축(ISO)",
            "불량 분석", "고객 품질 대응", "신뢰성 시험", "계측 관리", "TQM(전사적 품질 관리)"
        ],
        "공정 엔지니어": [
            "반도체 공정", "디스플레이 공정", "화학 공정", "제조 기술", "수율 개선",
            "공정 최적화", "Trouble Shooting", "장비 운영/관리", "센서/계측 제어", "생산 자동화 기술"
        ],
        "설비/기술 엔지니어": [
            "설비 유지보수(PM)", "장비 설치/셋업", "자동화 설비 관리", "전기/전자 설비", "유틸리티 관리",
            "기계 설비", "공무(건물/장비)", "공정 기술 지원", "에너지 관리", "안전 설비 관리"
        ],
        "연구/기술직": [
            "제품 개발(R&D)", "신소재 개발", "배합/합성 연구", "설계 엔지니어", "분석/시험",
            "파일럿 생산", "기술 이전", "기술 표준화", "특허 출원 지원", "연구 행정"
        ],
        "안전/환경": [
            "안전 관리자", "보건 관리자", "산업 안전", "위험성 평가", "환경 관리",
            "폐기물 처리", "대기/수질 오염 방지", "소방 안전", "EHS(환경안전보건)", "인증 대응"
        ],
        "화학/섬유/식품": [
            "화학 분석", "섬유 가공", "식품 공학", "제약 생산", "생물 공정",
            "발효/추출 기술", "품질 위생 관리", "신제품 레시피 개발", "HACCP/GMP 관리", "원료 분석"
        ],
        "조선/항공/플랜트": [
            "선박 설계", "항공기 부품 제조", "플랜트 시공 관리", "발전소 운영", "건설 엔지니어링",
            "현장 관리", "배관 설계", "기계 설계", "토목/건축 엔지니어링", "EPC 프로젝트 관리"
        ],
        "생산 오퍼레이터": [
            "설비 운전", "기계 조작", "CNC/머시닝 센터", "사출 성형", "용접/제관",
            "조립/가공", "제품 검사", "포장/출하", "크린룸 작업", "라인 관리"
        ],
        "기술 교육/지원": [
            "기술 매뉴얼 제작", "기술 교육", "AS/기술 지원", "고객 기술 상담", "제품 사용 교육",
            "현장 기술 전수", "기술 컨설팅", "기술 자료 작성", "교육 프로그램 개발", "품질 기술 지원"
        ]
    },
    # ====================================================================
    # 8. 무역/물류/유통 (SCM, 구매, MD 등)
    # ====================================================================
    "무역/물류/유통": {
        "물류 관리": [
            "SCM(공급망 관리)", "물류 계획", "창고 관리(WMS)", "운송 관리(TMS)", "재고 관리",
            "물류 시스템 운영", "3PL/4PL 관리", "물류비 절감", "포장/하역", "위험물 관리"
        ],
        "무역/수출입": [
            "수출입 사무", "통관", "관세사", "무역 서류 작성", "L/C/T/T 결제",
            "무역 계약", "해외 영업 지원", "바이어 관리", "무역 보험", "원산지 관리"
        ],
        "구매/자재": [
            "전략 구매(Sourcing)", "자재 관리", "외주 관리", "협력업체 관리", "원가 분석",
            "공급망 리스크 관리", "구매 시스템(SRM) 운영", "구매 계약", "재고 최적화", "MRO 구매"
        ],
        "유통/MD": [
            "상품 기획(MD)", "바잉(Buying)", "유통 채널 관리", "상품 소싱", "재고 운영",
            "가격 책정", "매출 분석", "브랜드 MD", "패션 MD", "온라인 MD"
        ],
        "택배/배송": [
            "배송 기사", "물류 센터 운영", "터미널 관리", "배송 경로 최적화", "라스트 마일 배송",
            "냉장/냉동 물류", "차량 관리", "퀵 서비스", "배송 시스템 운영", "허브 관리"
        ],
        "통관/운송": [
            "국제 운송", "포워딩", "해상/항공 운송", "통관 업무", "관세 환급",
            "운송 보험", "운송 계약", "위험물 운송", "컨테이너 관리", "복합 운송"
        ],
        "유통 매장 관리": [
            "매장 매니저", "판매 관리", "재고 관리", "VMD", "고객 서비스",
            "POS 시스템", "점포 개발", "시설 관리", "인력 관리", "매출 분석"
        ],
        "해외 소싱": [
            "해외 공급처 발굴", "품질 검수", "납기 관리", "소싱 계약", "원자재 수입",
            "해외 전시회 참가", "국제 무역법", "환율 리스크 관리", "샘플 관리", "공급망 감사"
        ],
        "SCM 시스템": [
            "ERP SCM 모듈 운영", "WMS/TMS 시스템", "물류 IT 기획", "시스템 구축 PM", "데이터 연동",
            "시스템 유지보수", "자동화 시스템 연동", "AI 기반 SCM", "수요 예측 시스템", "시스템 분석/설계"
        ],
        "농수산/식자재 유통": [
            "농산물 유통", "수산물 유통", "식자재 MD", "콜드체인 관리", "산지 직거래",
            "식자재 구매", "도매 시장 관리", "식품 안전 관리", "급식 운영 관리", "경매/입찰"
        ]
    },
    # ====================================================================
    # 9. 서비스/교육 (CS, 강사, 여행, 엔터테인먼트 등)
    # ====================================================================
    "서비스/교육": {
        "고객 서비스(CS)": [
            "고객 상담(인바운드)", "아웃바운드 상담", "VOC 분석", "CS 매니저", "서비스 교육",
            "콜센터 관리", "챗봇/AI 상담", "고객 만족도 조사", "민원 처리", "서비스 품질 관리"
        ],
        "교육 기획/운영": [
            "교육 프로그램 기획", "이러닝 콘텐츠 개발", "교육 평가", "학사/교무 행정", "직무 교육",
            "강사 섭외/관리", "교육 시설 관리", "평생 교육사", "기업 연수 기획", "HRD 전문가"
        ],
        "강사/교사": [
            "초/중/고 교사", "대학교수", "입시 컨설턴트", "학원 강사", "기업 교육 강사",
            "외국어 강사", "예체능 강사", "특수 교육 교사", "코치", "튜터"
        ],
        "항공/여행/숙박": [
            "승무원(스튜어디스)", "항공 예약/발권", "여행 상품 기획(MD)", "가이드/인솔자", "호텔리어",
            "컨벤션/이벤트", "리조트/레저 관리", "프런트 데스크", "투어 오퍼레이터", "객실 관리"
        ],
        "금융 서비스": [
            "은행 창구 업무", "텔러", "대출 상담", "보험 설계사", "PB(프라이빗 뱅킹)",
            "카드 발급/관리", "금융 상품 안내", "금융 콜센터", "리스크 관리 지원", "증권 계좌 관리"
        ],
        "의료/보건 서비스": [
            "간호사", "간호조무사", "의료 기사", "물리치료사", "병원 코디네이터",
            "의료 행정", "요양 보호사", "정신 건강 상담사", "임상 시험 코디네이터(CRC)", "응급 구조사"
        ],
        "사회 복지/상담": [
            "사회 복지사", "청소년 상담사", "직업 상담사", "심리 치료사", "요양원/복지관 운영",
            "사례 관리", "보호 관찰관", "가정 폭력 상담", "정신 건강 상담", "지역 사회 개발"
        ],
        "레저/스포츠": [
            "스포츠 강사", "헬스 트레이너", "요가/필라테스 강사", "수영장 관리", "경기장 운영",
            "스포츠 이벤트 기획", "스포츠 마케팅", "심판/감독", "레저 시설 관리", "재활 트레이너"
        ],
        "미디어/엔터테인먼트": [
            "PD(방송/영상)", "작가", "아나운서/기자", "연예인 매니저", "콘텐츠 기획",
            "음향/조명 엔지니어", "방송 기술", "공연 기획", "MCN/크리에이터 관리", "스튜디오 운영"
        ],
        "기타 서비스": [
            "통번역 서비스", "통신 서비스 개통", "미용/피부 관리사", "웨딩 플래너", "장례 지도사",
            "경비/보안", "청소/미화", "방문 요양", "주차 관리", "환경 미화"
        ]
    },
    # ====================================================================
    # 10. 건설/부동산 (시공, 설계, 개발, 관리 등)
    # ====================================================================
    "건설/부동산": {
        "건설 시공": [
            "건축 시공", "토목 시공", "플랜트 시공", "안전 관리", "현장 관리",
            "공정 관리", "품질 관리", "견적/원가 관리", "감리/감독", "자재 관리"
        ],
        "건축/설계": [
            "건축 설계", "인테리어 설계", "구조 설계", "기계 설비 설계", "전기 설비 설계",
            "소방 설비 설계", "CAD/BIM 모델링", "친환경 건축", "건축 자재 연구", "도시 계획"
        ],
        "부동산 개발": [
            "시행(Developer)", "PM(프로젝트 관리)", "타당성 분석", "부지 매입", "사업 인허가",
            "PF(프로젝트 금융)", "마케팅/분양", "자산 운용", "리스크 관리", "개발 컨설팅"
        ],
        "부동산 관리": [
            "시설 관리(FM)", "건물 관리", "임대차 관리", "자산 관리(Property Management)", "주택 관리사",
            "보안/경비 관리", "미화/청소 관리", "에너지 관리", "커뮤니티 관리", "임차인 서비스"
        ],
        "주택/건설 영업": [
            "아파트/상가 분양", "건설 자재 영업", "건설 장비 렌탈", "부동산 중개", "건설 프로젝트 영업",
            "건축 설계 영업", "인테리어/가구 영업", "해외 건설 수주 영업", "공공기관 영업", "시행사 영업"
        ],
        "토목/도시 계획": [
            "토목 설계", "구조 해석", "도시 계획", "교통 엔지니어링", "환경 영향 평가",
            "GIS/측량", "도로/교량 설계", "상하수도 설비", "국토 개발 계획", "지반 조사"
        ],
        "조경/환경": [
            "조경 설계", "식재/시설 시공", "공원 관리", "환경 엔지니어링", "생태 복원",
            "수질/대기 환경", "폐기물 처리 기술", "토양 오염 정화", "환경 영향 평가", "조경 감리"
        ],
        "감정평가/중개": [
            "감정평가사", "부동산 중개사", "자산 가치 평가", "공시지가 산정", "부동산 리서치",
            "투자 자문", "경매/공매 컨설팅", "부동산 금융 컨설팅", "재개발/재건축 분석", "부동산 세무"
        ],
        "건설 기계/장비": [
            "건설 기계 운전(포크레인 등)", "중장비 정비", "건설 장비 관리", "타워크레인 운전", "기계 설비 시공",
            "장비 임대 관리", "특수 장비 운영", "건설 기계 교육", "굴삭기 운전", "차량 정비"
        ],
        "해외 건설": [
            "해외 플랜트 건설", "해외 현장 관리", "국제 계약", "해외 자재 조달", "현지 인력 관리",
            "해외 금융 조달", "해외 영업", "기술 이전", "EPC 프로젝트 관리", "해외 공무"
        ]
    }
}

# 직무 중분류 업데이트 함수
def update_job_middle(major_category):
    """대분류 선택에 따라 중분류 드롭다운 옵션을 업데이트합니다."""
    if major_category in JOB_CATEGORIES:
        # 중분류 옵션 리스트 (딕셔너리의 키)
        middle_options = list(JOB_CATEGORIES[major_category].keys())
        
        # 첫 번째 중분류를 기본값으로 설정
        default_middle = middle_options[0] if middle_options else None
        
        # 중분류 업데이트 객체 반환
        return gr.update(choices=middle_options, value=default_middle), default_middle
    else:
        # 선택지가 없을 경우 빈 목록 반환
        return gr.update(choices=[], value=None), None

# 직무 소분류 업데이트 함수
def update_job_minor(major_category, middle_category):
    """대분류와 중분류 선택에 따라 소분류 드롭다운 옵션을 업데이트합니다."""
    if major_category in JOB_CATEGORIES and middle_category in JOB_CATEGORIES[major_category]:
        # 소분류 옵션 리스트 (중분류 딕셔너리의 값)
        minor_options = JOB_CATEGORIES[major_category][middle_category]
        
        # 첫 번째 소분류를 기본값으로 설정
        default_minor = minor_options[0] if minor_options else None
        
        # 소분류 업데이트 객체 반환
        return gr.update(choices=minor_options, value=default_minor)
    else:
        return gr.update(choices=[], value=None)
    

with gr.Blocks() as demo:
    gr.Markdown("# 👔 AI 면접 모의고사 & 피드백 시스템")
    gr.Markdown("직무 정보를 입력하면 AI가 예상 질문을 생성하고, 음성 답변을 분석해 피드백을 제공합니다.")

    # --- [섹션 1] 사용자 정보 입력 ---
    with gr.Row():
        with gr.Column(scale=1):
            # ... (career_type 및 years_input 생략) ...
            career_type = gr.Dropdown(["신입", "경력"], label="경력 구분", value="신입")
            years_input = gr.Textbox(label="연차 (년)", visible=False, placeholder="예: 3")
            
            def toggle_years(choice):
                return gr.update(visible=(choice == "경력"))
            career_type.change(toggle_years, inputs=career_type, outputs=years_input)
        
        with gr.Column(scale=1):
            salary = gr.Dropdown(["3,000~4,000만 원", "4,000~5,000만 원", "5,000만 원 이상", "면접 후 협의"], label="희망 연봉")

    with gr.Row():
        with gr.Column(scale=0.333):
            job_major = gr.Dropdown(list(JOB_CATEGORIES.keys()), label="직무 대분류", value="IT/개발")
        with gr.Column(scale=0.333):
            # 초기값은 IT/개발의 중분류로 설정
            initial_middle_choices = list(JOB_CATEGORIES["IT/개발"].keys())
            job_middle = gr.Dropdown(initial_middle_choices, label="직무 중분류", value=initial_middle_choices[0])
        with gr.Column(scale=0.333):
            # 초기값은 IT/개발 > 백엔드의 소분류로 설정
            initial_minor_choices = JOB_CATEGORIES["IT/개발"][initial_middle_choices[0]]
            job_minor = gr.Dropdown(initial_minor_choices, label="직무 소분류", value=initial_minor_choices[0])    

     # 1. 대분류 변경 이벤트
    job_major.change(
        fn=update_job_middle,
        inputs=[job_major],
        outputs=[job_middle, job_middle] # 중분류 드롭다운을 업데이트하고, 업데이트된 값을 다시 출력 (Chain 처리)
    )

    # 1-1. 대분류 변경 시 중분류가 업데이트 된 후, 소분류도 업데이트 되도록 Chain 연결
    job_major.change(
        fn=lambda m: update_job_minor(m, list(JOB_CATEGORIES[m].keys())[0]),
        inputs=[job_major],
        outputs=[job_minor]
    )


    # 2. 중분류 변경 이벤트
    # 중분류가 변경되면, 대분류 값과 새로 선택된 중분류 값을 함께 사용하여 소분류를 업데이트합니다.
    job_middle.change(
        fn=update_job_minor,
        inputs=[job_major, job_middle], # inputs에 job_major 값도 필요합니다.
        outputs=[job_minor]
    )       
            
    btn_generate = gr.Button("🚀 면접 질문 생성하기", variant="primary")

    kind = ["기술", "인성", "기술", "인성", "기술"]
    
    question_slots = [] # 슬롯들을 담을 리스트

    for i in range(5):
        with gr.Group() as slot_group:
            gr.Markdown(f"### ❓ {kind[i]} 질문 {i+1}")
            
            with gr.Row():
                # 질문 텍스트와 오디오
                q_text = gr.Markdown("질문 내용이 여기에 표시됩니다.")
            with gr.Row():
                q_audio = gr.Audio(label="질문 듣기", type="filepath", interactive=False)
            
            with gr.Row():
                # 사용자 답변 녹음
                # type="filepath"는 오디오 파일을 임시 경로로 저장하여 함수에 전달함
                user_mic = gr.Audio(sources=["microphone"], type="filepath", label="🎤 답변 녹음하기")
            
            btn_analyze = gr.Button("📝 답변 제출 및 분석('답변 제출 및 분석' 버튼 클릭 후, 잠시 대기해주시기 바랍니다. [최대 20초])", size="md")
            
            # 분석 결과 표시 영역
            feedback_display = gr.Markdown("")

            # 이 슬롯의 컴포넌트들을 딕셔너리로 저장해둠
            question_slots.append({
                "group": slot_group,
                "q_text": q_text,
                "q_audio": q_audio,
                "user_mic": user_mic,
                "btn_analyze": btn_analyze,
                "feedback": feedback_display,
                "q_content_state": gr.State() # 질문 텍스트 저장용 변수,
            })
           

    # [이벤트 연결] 각 슬롯의 분석 버튼 로직 연결
    for slot in question_slots:
        slot["btn_analyze"].click(
            fn=analyze_answer_logic,
            inputs=[slot["user_mic"], slot["q_content_state"]],
            outputs=[slot["feedback"]]
        )

    # --- [메인 로직] 질문 생성 버튼 클릭 시 실행될 함수 ---
    def process_generation(career, yr, maj, mid, min, sal):
             
        updates = []
        
        for i in range(5):
            # 1. 질문 텍스트 생성 (OpenAI)
            questions = generate_questions_logic(career, yr, maj, mid, min, sal, kind[i])
            
            # 2. 각 질문에 대해 TTS 생성 및 UI 업데이트 리스트 구성
            # 질문이 존재하는 슬롯
            q_str = questions[0]
            audio_path = text_to_speech_file(q_str) # TTS 변환
            
            updates.append(gr.update(visible=True)) # Group visible
            updates.append(gr.update(value=f"**{q_str}**")) # Markdown Text
            updates.append(gr.update(value=audio_path)) # Audio File
            updates.append(q_str) # State (질문 내용 저장)
            updates.append(gr.update(interactive=True)) # Mic 초기화
            updates.append(gr.update(value="")) # Feedback 초기화

        return updates

    # [이벤트 연결] 생성 버튼 -> process_generation -> 각 슬롯 업데이트
    # outputs 리스트는 위에서 append한 순서와 정확히 일치해야 함
    output_components = []
    for slot in question_slots:
        output_components.append(slot["group"])
        output_components.append(slot["q_text"])
        output_components.append(slot["q_audio"])
        output_components.append(slot["q_content_state"])
        output_components.append(slot["user_mic"])
        output_components.append(slot["feedback"])
       
    btn_generate.click(
        fn=process_generation,
        inputs=[career_type, years_input, job_major, job_middle, job_minor, salary],
        outputs=output_components
    )

# 실행
if __name__ == "__main__":
    demo.launch(share=True)
    #demo.launch()

c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\layouts\column.py:59: UserWarning: 'scale' value should be an integer. Using 0.333 will cause issues.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7882
* Running on public URL: https://2a879830766c022197.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


response_json : {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'annotations': [], 'content': 'JPA를 사용할 때 연관관계 맵핑에서 FetchType.EAGER와 FetchType.LAZY를 사용할 경우 각각의 장단점과 사용할 때 주의해야 할 점을 설명해주세요.', 'refusal': None, 'role': 'assistant'}}], 'created': 1764925867, 'id': 'chatcmpl-CjM8BYuybJb8L5pmssmQvYdnlnFF8', 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filt

Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\proactor_ev

response_json : {'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'annotations': [], 'content': 'JDBC를 이용해서 데이터베이스와의 연결을 처리할 때, 커넥션 풀을 사용하는 이유와 이를 사용하지 않을 경우 발생할 수 있는 문제점은 무엇인지 설명해주세요.', 'refusal': None, 'role': 'assistant'}}], 'created': 1764925903, 'id': 'chatcmpl-CjM8l5Xp8YdQZzkP5tnuaASFl1QTI', 'model': 'gpt-4o-mini-2024-07-18', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered':

Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] 현재 연결은 원격 호스트에 의해 강제로 끊겼습니다
ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EL32\AppData\Local\Programs\Python\Python311\Lib\site-packages\uvicorn\m